# TPOT

Fuente: https://raw.githubusercontent.com/rpi-techfundamentals/fall2018-materials/master/fig/final-logo.png)](http://rpi.analyticsdojo.com
https://towardsdatascience.com/automated-machine-learning-on-the-cloud-in-python-47cf568859f

# Introduction: Automated Machine Learning

En este Notebook, veremos cómo usar [TPOT] (https://epistasislab.github.io/tpot/api/), una libreria de Python que permite el preprocesamiento automático, selección de modelos y ajuste de hiperparámetros. Utilizando [programación genética] (http://geneticprogramming.com/tutorial/), TPOT intenta encontrar la mejor canalización de aprendizaje automático para un conjunto de datos evaluando miles de posibilidades.

Los pasos de aprendizaje automático en este contexto consisten en:

1. Preprocesamiento de funciones
  * Imputar valores perdidos y escalar valores
  * Construir nuevas características como transformaciones polinomiales
2. Selección de funciones
  * Reducción de dimensionalidad, por ejemplo usando PCA y otras técnicas
3. Selección de modelo
  * Evaluación de varios modelos de aprendizaje automático
4. Ajuste de hiperparámetros
  * Encontrar la configuración óptima del modelo para el problema particular

TPOT pertenece a una clase de métodos conocidos como [auto-ml (abreviatura de aprendizaje automático automatizado)] (https://www.kdnuggets.com/2017/01/current-state-automated-machine-learning.html) que tienen como objetivo simplificar el trabajo del científico de datos al encontrar automáticamente los pasos de preprocesamiento de características óptimos y el modelo. 

El aprendizaje automático suele ser una parte de un problema de ciencia de datos que requiere mucho tiempo y mucho conocimiento. Auto-ml no está diseñado para reemplazar al científico de datos, sino para liberarlo para trabajar en aspectos más importantes del problema completo, como adquirir datos e interpretar los resultados del modelo. 

Otras fuentes de modelos de AutoML:

* [Auto-sklearn](https://automl.github.io/auto-sklearn/stable/)
* [H20](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html)
* [Google Cloud AutoML](https://cloud.google.com/automl/)



## Instalación de Tpot

In [1]:
# Install tpot on the server
!pip install tpot

In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Import the tpot regressor
from tpot import TPOTRegressor

C:\Users\Leyre\anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


# Descripción del problema

Se trata de un problema de regresión supervisada: datos [datos de energía de la ciudad de Nueva York] (http://www.nyc.gov/html/gbee/html/plan/ll84_scores.shtml), queremos construir un modelo que pueda predecir el Energy Star Score de un edificio. Utilizaremos la ingeniería de características manual, la reducción de dimensionalidad, la selección del modelo y el ajuste de hiperparámetros pra tal fin.

## Conjunto de datos

Las características contienen una serie de variables numéricas continuas (como el uso de energía y el área del edificio), así como dos variables categóricas codificadas en caliente (municipio y tipo de edificio). Hay un total de 82 funciones.

Todos los valores perdidos se han codificado como "np.nan" y TPOT realizará automáticamente la imputación del valor perdido. También escala automáticamente las variables para que no tengamos que preocuparnos por normalizar el rango de cada característica. TPOT realiza tanto ingeniería de características como selección de características, por lo que no transformaremos ninguna de las variables ni eliminaremos características extrañas que pensamos que pueden ser extrañas.

In [3]:
# Read in features from GitHub
train_features = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/X_train.csv')
test_features = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/X_test.csv')

# Read in labels from GitHub
train_labels = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/Y_train.csv')
test_labels = pd.read_csv('https://raw.githubusercontent.com/WillKoehrsen/machine-learning-project-walkthrough/master/data/Y_test.csv')

print('Training features shape: ', train_features.shape)
print('Testing features shape:  ', test_features.shape)

Training features shape:  (6622, 82)
Testing features shape:   (2839, 82)


In [4]:
train_features.head()

,Order,Property Id,DOF Gross Floor Area,Largest Property Use Type - Gross Floor Area (ft²),Year Built,Number of Buildings - Self-reported,Occupancy,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Weather Normalized Site Electricity Intensity (kWh/ft²),...,Largest Property Use Type_Restaurant,Largest Property Use Type_Retail Store,Largest Property Use Type_Self-Storage Facility,Largest Property Use Type_Senior Care Community,Largest Property Use Type_Social/Meeting Hall,Largest Property Use Type_Strip Mall,Largest Property Use Type_Supermarket/Grocery Store,Largest Property Use Type_Urgent Care/Clinic/Other Outpatient,Largest Property Use Type_Wholesale Club/Supercenter,Largest Property Use Type_Worship Facility
0,13276,5849784,90300.0,77300.0,1950,1,100,126.0,136.8,5.2,...,0,0,0,0,0,0,0,0,0,0
1,7377,4398442,52000.0,52000.0,1926,1,100,95.4,102.0,4.7,...,0,0,0,0,0,0,0,0,0,0
2,9479,4665374,104700.0,105000.0,1954,1,100,40.4,40.0,3.8,...,0,0,0,0,0,0,0,0,0,0
3,14774,3393340,129333.0,129333.0,1992,1,100,157.1,163.1,16.9,...,0,0,0,1,0,0,0,0,0,0
4,3286,2704325,109896.0,116041.0,1927,1,100,62.3,68.2,3.5,...,0,0,0,0,0,0,0,0,0,0


En el siguiente código, convertimos a matrices numpy. Esto no es estrictamente necesario, pero las etiquetas deben convertirse a un vector unidimensional (usando remodelar en el código a continuación) o Scikit-Learn mostrará un mensaje de advertencia.

In [5]:
# Convert to numpy arrays
training_features = np.array(train_features)
testing_features = np.array(test_features)

# Sklearn wants the labels as one-dimensional vectors
training_targets = np.array(train_labels).reshape((-1,))
testing_targets = np.array(test_labels).reshape((-1,))

Después de la preparación mínima de datos, podemos crear el optimizador TPOT. La sintaxis de los optimizadores TPOT (https://epistasislab.github.io/tpot/using/#tpot-with-code) está diseñada para ser lo más cercana posible a la de los modelos Scikit-Learn.

Los [parámetros predeterminados para los optimizadores de TPOT] (https://epistasislab.github.io/tpot/api/) probarán 100 poblaciones de tuberías, cada una con 100 generaciones para un total de 10,000 tuberías. Con una validación cruzada de 10 veces, esto representa 100.000 ejecuciones de entrenamiento. Esto lleva bastante tiempo. Para evitar quedarse sin tiempo estableceremos un máximo de 15 minutos para la evaluación. [TPOT está diseñado para ejecutarse durante días] (https://epistasislab.github.io/tpot/using/) para evaluar a fondo muchas canalizaciones, pero los resultados pueden ser bastante buenos incluso con unas pocas horas de entrenamiento.

Establecemos los siguientes parámetros en la llamada al optimizador (no dude en cambiarlos y ver cómo afectan los resultados):

* `scoring = neg_mean_absolute_error`: Nuestra métrica de rendimiento de regresión seleccionada
* `max_time_mins = 15`: Limite la evaluación a 15 minutos
* `n_jobs = -1`: Usa todos los núcleos disponibles en la máquina
* `verbosity = 2`: muestra una cantidad limitada de información durante el entrenamiento
* `cv = 5`: Utilice una validación cruzada de 5 veces (el valor predeterminado es 10)

Después de crear el optimizador, lo "ajustamos" a los datos de entrenamiento como con cualquier modelo de aprendizaje automático de Scikit-Learn. Esto inicia el proceso de optimización que continuará durante 8 horas. Durante el entrenamiento, podemos ver una cantidad limitada de información (cambie la "verbosidad" para ver más o menos)

In [6]:
# Create a tpot object with a few parameters
tpot = TPOTRegressor(scoring = 'neg_mean_absolute_error', 
                    max_time_mins = 15, 
                    n_jobs = -1,
                    verbosity = 2,
                    cv = 5)

In [7]:
# Fit the tpot model on the training data
tpot.fit(training_features, training_targets)

Imputing missing values in feature set



Generation 1 - Current best internal CV score: -8.70977287835271
Generation 2 - Current best internal CV score: -8.70977287835271
15.21 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBRegressor(LassoLarsCV(input_matrix, normalize=True), learning_rate=0.1, max_depth=5, min_child_weight=19, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.5)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=100,
              log_file=<ipykernel.iostream.OutStream object at 0x000002DE9B11F9C8>,
              max_eval_time_mins=5, max_time_mins=15, memory=None,
              mutation_rate=0.9, n_jobs=-1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=None, scoring='neg_mean_absolute_error',
              subsample=1.0, template=None, use_dask=False, verbosity=2,
              warm_start=False)

Debido al límite de tiempo, podemos ver que nuestro modelo solo pudo pasar 15 generaciones. Con 100 poblaciones, esto representa 1500 tuberías individuales diferentes que fueron evaluadas, ¡deje algunas más de las que podríamos probar a mano!

Una vez que el modelo ha terminado de entrenarse, podemos ver la canalización óptima imprimiendo la "tubería_ ajustada". Esto representa la canalización completa con la mejor métrica de rendimiento (en este caso, el `neg_mean_absolute_error` más alto) de la validación cruzada.

In [8]:
# Show the final model
print(tpot.fitted_pipeline_)

Pipeline(memory=None,
         steps=[('stackingestimator',
                 StackingEstimator(estimator=LassoLarsCV(copy_X=True, cv=None,
                                                         eps=2.220446049250313e-16,
                                                         fit_intercept=True,
                                                         max_iter=500,
                                                         max_n_alphas=1000,
                                                         n_jobs=None,
                                                         normalize=True,
                                                         positive=False,
                                                         precompute='auto',
                                                         verbose=False))),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                  

El proceso de optimización de TPOT es estocástico, lo que significa que [cada ejecución producirá resultados diferentes] (https://epistasislab.github.io/tpot/using/). Si vuelve a ejecutar este cuaderno, no se preocupe si ve una canalización final diferente.

Para guardar la canalización para uso futuro, podemos exportarla a una secuencia de comandos de Python.

In [9]:
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline.py')

Si queremos ver todas las canalizaciones evaluadas, podemos ver el atributo `.evaluate_individuals_` del optimizador ajustado. 

In [ ]:
# To examine all fitted models
tpot.evaluated_individuals_

Finalmente, probemos toda la tubería ajustada en el conjunto de datos de prueba. Después de evaluar todos los pipelines, TPOT guarda el mejor y lo entrena con todos los datos de entrenamiento, para que podamos evaluar el mejor utilizando el método optimizador .score. Esto mostrará el error cuadrático medio negativo, nuestra métrica de regresión.

In [ ]:
# Evaluate the final model
print(tpot.score(testing_features, testing_targets))

In [ ]:
#Para guardar el codigo
# Download the pipeline for local use
files.download('tpot_exported_pipeline.py')

Aquí está el codigo final del modelo, del archivo guardado de Python descargado. ¡Podemos entrenarlo y probarlo solo para asegurarnos de que esta puntuación sea correcta!

In [ ]:
# Imports that the final pipeline needs
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer
from tpot.builtins import StackingEstimator

# Preprocessing steps
imputer = Imputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Final pipeline from TPOT
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    GradientBoostingRegressor(alpha=0.95, learning_rate=0.1, loss="lad", 
                              max_depth=7, max_features=0.75, 
                              min_samples_leaf=3, min_samples_split=18, 
                              n_estimators=100, subsample=0.60)
)

In [ ]:
# Fit on the training data
exported_pipeline.fit(training_features, training_targets)

Después de crear la canalización optimizada y entrenarla, podemos evaluarla en el conjunto de pruebas. Como los modelos no se crearon con un "estado_aleatorio", esperamos un rendimiento ligeramente diferente al de los resultados originales, pero debería ser bastante similar.

In [ ]:
# Make predictions on the testing data
predictions = exported_pipeline.predict(testing_features)

print('Mean Absolute Error = %0.4f' % np.mean(abs(predictions - testing_targets)))

Efectivamente, el error absoluto medio está cerca del del método optimizador `.score` y considerablemente mejor que nuestros esfuerzos manuales de construcción de canalizaciones.

A partir de aquí, podemos usar los resultados de optimización e intentar ajustar aún más la canalización, o podemos pasar a fases importantes del flujo de trabajo de ciencia de datos. Si usamos esto como modelo final, podríamos dedicar tiempo a tratar de interpretar el modelo (tal vez usando [LIME: Local Interpretable Model-Agnostic Explainations] (https://www.oreilly.com/learning/introduction-to-local- interpretable-model-agnostic-explications-lime)) o informando nuestros resultados.